### Prerequisites

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/'My Drive'/pets/YOLO_test/

/content/gdrive/My Drive/pets/YOLO_test


In [ ]:
!git clone https://github.com/WongKinYiu/yolov7

In [ ]:
%%bash
pip install -r yolov7/requirements.txt
pip install fiftyone
pip install wandb

### Data analysis using fiftyone

In [ ]:
import os
import fiftyone as fo

name = "yolo-dataset"

try:
    dataset = fo.Dataset(name)

    dataset_dir = os.getcwd()
    splits = ["train", "val"]

    for split in splits:
        dataset.add_dir(
            dataset_dir=dataset_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            split=split,
            tags=split,
    )
except Exception:
    print('This dataset already exists')
    fo.load_dataset(name)

print(dataset)
print(dataset.head())

Migrating database to v0.17.2


INFO:fiftyone.migrations.runner:Migrating database to v0.17.2


 100% |███████████████| 2577/2577 [13.7m elapsed, 0s remaining, 5.0 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 2577/2577 [13.7m elapsed, 0s remaining, 5.0 samples/s]      


 100% |█████████████████| 645/645 [3.4m elapsed, 0s remaining, 3.8 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 645/645 [3.4m elapsed, 0s remaining, 3.8 samples/s]      


Name:        yolo-dataset
Media type:  image
Num samples: 3222
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
[<Sample: {
    'id': '63561a2063b8523b3d4c75e1',
    'media_type': 'image',
    'filepath': '/content/gdrive/My Drive/pets/YOLO_test/train/images/00000001.jpg',
    'tags': ['train'],
    'metadata': None,
    'ground_truth': <Detections: {'detections': []}>,
}>, <Sample: {
    'id': '63561a2163b8523b3d4c75eb',
    'media_type': 'image',
    'filepath': '/content/gdrive/My Drive/pets/YOLO_test/train/images/00000002.jpg',
    'tags': ['train'],
    'metadata': None,
    'ground_truth': <Detections: {
 

### Data visualization

In [ ]:
fo.launch_app(dataset)

Dataset:          yolo-dataset
Media type:       image
Num samples:      3222
Selected samples: 0
Selected labels:  0
Session type:     colab

### Training stage

In [ ]:
%cd yolov7/

In [ ]:
!python3 train.py --weights weights/yolov7_training.pt --data data/custom.yaml --workers 4 --batch-size 4 --img 416 --cfg cfg/training/yolov7.yaml --name yolov7 --hyp data/hyp.scratch.p5.yaml --epochs 50

YOLOR 🚀 2022-10-21 torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=4, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/yolov7.yaml', data='data/custom.yaml', device='', entity=None, epochs=50, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[416, 416], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov7', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov73', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=4, upload_dataset=False, v5_metric=False, weights='weights/yolov7_training.pt', workers=4, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.

### Evaluating the model

You can see all the charts and metrics in readme.md

In [ ]:
!python3 test.py --data data/custom.yaml --img 416 --batch 32 --conf 0.001 --iou 0.65 --device 0 --weights runs/train/yolov73/weights/best.pt --name yolov7_416_val

Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/custom.yaml', device='0', exist_ok=False, img_size=416, iou_thres=0.65, name='yolov7_416_val', no_trace=False, project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', v5_metric=False, verbose=False, weights=['runs/train/yolov73/weights/best.pt'])
YOLOR 🚀 2022-10-21 torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36492560 parameters, 6194944 gradients, 103.2 GFLOPS
 Convert model to Traced-model... 
 traced_script_

### Visualizing object detection

You can see the results in readme.md

In [ ]:
!python3 detect.py --weights runs/train/yolov73/weights/best.pt --conf 0.25 --img-size 416 --source ../val/images/1192.jpg

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=416, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='../val/images/1192.jpg', update=False, view_img=False, weights=['runs/train/yolov73/weights/best.pt'])
YOLOR 🚀 2022-10-21 torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36492560 parameters, 6194944 gradients, 103.2 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 u